In [1]:
from nltk.util import pr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Anastasia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv("/Users/Anastasia/Downloads/labeled_data.csv")



In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;


In [5]:
tweets = data[["tweet"]]
tweets

,tweet
0,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
...,...
24778,"you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls @corey_emanuel right! His TL is trash &#8230;. Now, mine? Bible scriptures and hymns&#8221;"
24779,"you've gone and broke the wrong heart baby, and drove me redneck crazy"
24780,young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again
24781,youu got wild bitches tellin you lies


In [6]:
#clean tweet data

def clean(text):
    text = str(text).lower()
    text = re.sub('', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [7]:


data["tweet"] = data['tweet'].apply(clean)



In [8]:
# remove stopwords

def preprocessing (text):
    text = [word for word in text.split() if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split()]
    text = " ".join(text)
    return text

In [9]:


data["tweet"] = data['tweet'].apply(clean)



In [10]:
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt mayasolovely as a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out
1,1,3,0,3,0,1,rt boy dats coldtyga dwn bad for cuffin dat hoe in the place
2,2,3,0,3,0,1,rt urkindofbrand dawg rt you ever fuck a bitch and she start to cry you be confused as shit
3,3,3,0,2,1,1,rt cganderson vivabased she look like a tranny
4,4,6,0,6,0,1,rt shenikaroberts the shit you hear about me might be true or it might be faker than the bitch who told it to ya


In [11]:
# divide into hate, offensive, neither

class_new = data["class"].map({0:1,1:0,2:0})
data["class"] = class_new
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,0,rt mayasolovely as a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out
1,1,3,0,3,0,0,rt boy dats coldtyga dwn bad for cuffin dat hoe in the place
2,2,3,0,3,0,0,rt urkindofbrand dawg rt you ever fuck a bitch and she start to cry you be confused as shit
3,3,3,0,2,1,0,rt cganderson vivabased she look like a tranny
4,4,6,0,6,0,0,rt shenikaroberts the shit you hear about me might be true or it might be faker than the bitch who told it to ya


In [12]:
categories = {1:"hate_speech",0:"another",}
value = data['class'].map(categories)

In [13]:
# evaluate if hate speech

index = data.columns.get_loc("tweet")
data.insert(index,"value",value)



In [14]:


data.head()



,Unnamed: 0,count,hate_speech,offensive_language,neither,class,value,tweet
0,0,3,0,0,3,0,another,rt mayasolovely as a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out
1,1,3,0,3,0,0,another,rt boy dats coldtyga dwn bad for cuffin dat hoe in the place
2,2,3,0,3,0,0,another,rt urkindofbrand dawg rt you ever fuck a bitch and she start to cry you be confused as shit
3,3,3,0,2,1,0,another,rt cganderson vivabased she look like a tranny
4,4,6,0,6,0,0,another,rt shenikaroberts the shit you hear about me might be true or it might be faker than the bitch who told it to ya
